In [2]:
!pip install contractions
!pip install fasttext
!pip install pyLDAvis

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
import string
import fasttext
import contractions
import matplotlib as plt
import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"atdeepanshu","key":"efb7cfbda38df5a60eec6e85835e4859"}'}

In [5]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [6]:
!kaggle competitions download -c fake-news


test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
submit.csv: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!ls

'kaggle (1).json'   sample_data   test.csv.zip
 kaggle.json	    submit.csv	  train.csv.zip


In [8]:
train_data = pd.read_csv('train.csv.zip')

In [9]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analysts a...",1
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilia...,1
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,Howard Portnoy,Print \nAn Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Gu...,1


In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [11]:
def pandas_profile(df):
    print(train_data.describe())
    print(train_data.info())
    from pandas_profiling import ProfileReport
    prof = ProfileReport(df)
    prof.to_file(output_file='report.html')

In [12]:
pandas_profile(train_data)

                 id         label
count  20800.000000  20800.000000
mean   10399.500000      0.500625
std     6004.587135      0.500012
min        0.000000      0.000000
25%     5199.750000      0.000000
50%    10399.500000      1.000000
75%    15599.250000      1.000000
max    20799.000000      1.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB
None


/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:256: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result = pd.Series({}, name=data.name)
/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:256: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result = pd.Series({}, name=data.name)
/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:256: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  result = pd.Series({}, name=data.name)
/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:256: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'flo

TypeError: ignored

In [ ]:
from IPython.display import HTML
report = './report.html'
HTML(filename=report)

In [13]:
train_data.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [15]:
def create_sequences(text):
    from tensorflow.keras.preprocessing.text import Tokenizer
    tok = Tokenizer()
    tok.fit_on_texts(text)
    sequence = tok.texts_to_sequences(text)
    return sequence
    

In [16]:
def clean_data(sentence):
    if isinstance(sentence, str):
        clean_sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
        return clean_sentence
    else:
        raise ValueError("error in sentence", sentence)

In [17]:
train_data.dropna(subset = ["text"], inplace=True)
train_data['text'].apply(clean_data)


0        House Dem Aide We Didn t Even See Comey s Letter Until Jason Chaffetz Tweeted It By Darrell Lucu...
1        Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...
2        Why the Truth Might Get You Fired October 29 2016 The tension between intelligence analysts and ...
3        Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilia...
4        Print An Iranian woman has been sentenced to six years in prison after Iran s Revolutionary Guar...
                                                        ...                                                 
20795    Rapper T I unloaded on black celebrities who met with Donald Trump after the election saying the...
20796    When the Green Bay Packers lost to the Washington Redskins in Week 11 dropping to Aaron Rodgers ...
20797    The Macy s of today grew from the union of several great names in American retailing including i...
20798    NATO Russi

In [18]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analysts a...",1
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilia...,1
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,Howard Portnoy,Print \nAn Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Gu...,1


In [19]:
def split_sentence(sentence):
    if isinstance(sentence, str):
        return sentence.split()

def join_sentence(sentence):
    return ' '.join(sentence)

In [20]:
train_data['text_split'] = train_data['text'].apply(split_sentence)

In [21]:
train_data['text_split']

0        [House, Dem, Aide:, We, Didn’t, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It,...
1        [Ever, get, the, feeling, your, life, circles, the, roundabout, rather, than, heads, in, a, stra...
2        [Why, the, Truth, Might, Get, You, Fired, October, 29,, 2016, The, tension, between, intelligenc...
3        [Videos, 15, Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified, The, rate, at...
4        [Print, An, Iranian, woman, has, been, sentenced, to, six, years, in, prison, after, Iran’s, Rev...
                                                        ...                                                 
20795    [Rapper, T., I., unloaded, on, black, celebrities, who, met, with, Donald, Trump, after, the, el...
20796    [When, the, Green, Bay, Packers, lost, to, the, Washington, Redskins, in, Week, 11,, dropping, t...
20797    [The, Macy’s, of, today, grew, from, the, union, of, several, great, names, in, American, retail...
20798    [NATO,, Ru

In [22]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
train_data['stopwords_removed'] = train_data['text_split'].apply(lambda x: [word for word in x if word not in stop_words])
train_data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,title,author,text,label,text_split,stopwords_removed
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1,"[House, Dem, Aide:, We, Didn’t, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It,...","[House, Dem, Aide:, We, Didn’t, Even, See, Comey’s, Letter, Until, Jason, Chaffetz, Tweeted, It,..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0,"[Ever, get, the, feeling, your, life, circles, the, roundabout, rather, than, heads, in, a, stra...","[Ever, get, feeling, life, circles, roundabout, rather, heads, straight, line, toward, intended,..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analysts a...",1,"[Why, the, Truth, Might, Get, You, Fired, October, 29,, 2016, The, tension, between, intelligenc...","[Why, Truth, Might, Get, You, Fired, October, 29,, 2016, The, tension, intelligence, analysts, p..."
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilia...,1,"[Videos, 15, Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified, The, rate, at...","[Videos, 15, Civilians, Killed, In, Single, US, Airstrike, Have, Been, Identified, The, rate, ci..."
4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,Howard Portnoy,Print \nAn Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Gu...,1,"[Print, An, Iranian, woman, has, been, sentenced, to, six, years, in, prison, after, Iran’s, Rev...","[Print, An, Iranian, woman, sentenced, six, years, prison, Iran’s, Revolutionary, Guard, searche..."


In [23]:
words = train_data['stopwords_removed']
allwords = []
for wordlist in words:
    allwords += wordlist

In [24]:
train_data['text_processed'] = train_data['stopwords_removed'].apply(join_sentence)

In [25]:
train_data['text_processed']

0        House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...
1        Ever get feeling life circles roundabout rather heads straight line toward intended destination?...
2        Why Truth Might Get You Fired October 29, 2016 The tension intelligence analysts political polic...
3        Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate civilians killed...
4        Print An Iranian woman sentenced six years prison Iran’s Revolutionary Guard searched home found...
                                                        ...                                                 
20795    Rapper T. I. unloaded black celebrities met Donald Trump election, saying failed challenge presi...
20796    When Green Bay Packers lost Washington Redskins Week 11, dropping Aaron Rodgers vowed “run table...
20797    The Macy’s today grew union several great names American retailing, including namesake chain, Bl...
20798    NATO, Russ

In [31]:
allwords

['House',
 'Dem',
 'Aide:',
 'We',
 'Didn’t',
 'Even',
 'See',
 'Comey’s',
 'Letter',
 'Until',
 'Jason',
 'Chaffetz',
 'Tweeted',
 'It',
 'By',
 'Darrell',
 'Lucus',
 'October',
 '30,',
 '2016',
 'Subscribe',
 'Jason',
 'Chaffetz',
 'stump',
 'American',
 'Fork,',
 'Utah',
 '(',
 'image',
 'courtesy',
 'Michael',
 'Jolley,',
 'available',
 'Creative',
 'Commons-BY',
 'license)',
 'With',
 'apologies',
 'Keith',
 'Olbermann,',
 'doubt',
 'Worst',
 'Person',
 'The',
 'World',
 'week–FBI',
 'Director',
 'James',
 'Comey.',
 'But',
 'according',
 'House',
 'Democratic',
 'aide,',
 'looks',
 'like',
 'also',
 'know',
 'second-worst',
 'person',
 'well.',
 'It',
 'turns',
 'Comey',
 'sent',
 'now-infamous',
 'letter',
 'announcing',
 'FBI',
 'looking',
 'emails',
 'may',
 'related',
 'Hillary',
 'Clinton’s',
 'email',
 'server,',
 'ranking',
 'Democrats',
 'relevant',
 'committees',
 'didn’t',
 'hear',
 'Comey.',
 'They',
 'found',
 'via',
 'tweet',
 'one',
 'Republican',
 'committee',
 'ch

In [33]:
mostcommon = FreqDist(allwords).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words', fontsize=100)
plt.tight_layout(pad=0)
plt.show()

In [27]:
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words', fontsize=60)
plt.show()

In [ ]:
group_by = train_data.groupby('label')['text_processed'].apply(lambda x: Counter(' '.join(x).split()).most_common(25))
group_by_0 = group_by.iloc[0]
words0 = list(zip(*group_by_0))[0]
freq0 = list(zip(*group_by_0))[1]
plt.figure(figsize=(50,30))
plt.bar(words0, freq0)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for not fake ', fontsize=60)
plt.show()

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=25, max_features=50000)
tf = tf_vectorizer.fit_transform(train_data['text_processed'].values.astype('U'))
tf_feature_names = tf_vectorizer.get_feature_names()
# doc_term_matrix = pd.DataFrame(tf.toarray(), columns=list(tf_feature_names))
# doc_term_matrix

In [ ]:
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=500, random_state=0).fit(tf)
no_top_words = 10
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                          for i in topic.argsort()[:-no_top_words - 1:-1]]))
              
display_topics(lda_model, tf_feature_names, no_top_words)

In [ ]:
group_by = train_data.groupby('label')['text_processed'].apply(lambda x: Counter(' '.join(x).split()).most_common(25))
group_by_0 = group_by.iloc[1]
words0 = list(zip(*group_by_0))[0]
freq0 = list(zip(*group_by_0))[1]
plt.figure(figsize=(50,30))
plt.bar(words0, freq0)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for fake', fontsize=60)
plt.show()

In [ ]:
def max_len(expressions):
    splitted_expressions = []
    splitted_expressions.extend(w.split(' ') for w in expressions)
    maxlen = len(max(splitted_expressions, key=len))
    return maxlen

In [ ]:
def prepare_X_y(df):
    df.dropna(subset = ["text"], inplace=True)
    df['text'].apply(clean_data)
    text = list(df['text'])
    sequences = create_sequences(text)
    maxlen = max_len(text)
    from keras.preprocessing import sequence
    X = sequence.pad_sequences(sequences, maxlen=maxlen, padding='post',
                                              truncating='post')
    y = df['label']
    y_train = np.asarray(y).astype('float32')
    return X , y_train
    
    

In [ ]:
X,y = prepare_X_y(train_data)

In [ ]:
len(X[2])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
X_train.shape

In [ ]:

X_train1 = np.expand_dims(X_train, 2)

In [ ]:
X_train1.shape

In [ ]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential() 
    model.add(layers.LSTM(512, input_shape=(24234, 1), return_sequences=True)) 
    model.add(layers.LSTM(128, return_sequences=False))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc']) 
    history = model.fit(X_train1, y_train, validation_data=(X_test, y_test),epochs=10, batch_size=12, verbose=1)
    return history

In [ ]:
history = build_model()


In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict['loss'] 
acc = history_dict['acc'] 
val_loss_values = history_dict['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss') 
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss') 
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf() 
acc_values = history_dict['acc'] 
val_acc_values = history_dict['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc_values, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.xlabel('Epochs')
plt.ylabel('Loss') 
plt.legend()
plt.show()